## 🌟 Scenario: Content Moderation System

### The Problem

You've joined a growing tech community platform that has **50,000 users** but only **3 moderators**:
- **Sarah** manually reviews posts (6 hours/day, 200+ posts in queue)
- **Mike** tries to help users improve content (rarely has time)
- **Lisa** identifies harmful content (can't keep up)

**Current Issues:**
- Takes 5-10 minutes per post to check safety, tone, and grammar manually
- Users don't understand why content is rejected
- No time to enhance approved content

### Your Solution

Build an **AI-Powered Content Moderation System** that:

1. **Classifies** content type (social media post / article / comment)
2. **Analyzes** safety, tone, and grammar **in parallel**
3. **Scores** and decides: approve or reject
4. **Enhances** approved content automatically
5. **Provides feedback** to users

**Expected Impact:** Reduce moderation time from 5-10 minutes to 30 seconds per post!

### Example Test Cases

Your system should handle:

**✅ Good Content (needs enhancement):**
```
just finished reading an amzing book about AI ethics! 
its really make me think about how we build responsible systems.
```
→ Approve, fix grammar, enhance

**⚠️ Problematic Content:**
```
I hate this stupid product! Complete waste of money.
```
→ Flag for aggressive language, suggest constructive rephrasing

---

## 📋 Challenge Overview

### Your Mission

Build an **AI-Powered Content Moderation & Enhancement System** that:
1. Analyzes user-submitted content (text posts)
2. Moderates for safety and quality
3. Provides improvement suggestions
4. Enhances approved content

### Why This Challenge?

This challenge combines **multiple agentic patterns** in a realistic scenario:
- **Routing**: Classify content type (social media post, article, comment)
- **Evaluator-Optimizer**: Assess content quality and iterate improvements
- **Parallelization**: Analyze multiple aspects simultaneously (tone, safety, grammar)
- **Orchestrator-Worker**: Coordinate the full moderation pipeline
- **Prompt Chaining**: Transform raw content through moderation → enhancement → finalization

---

## 🎓 Part 1: Framework Selection & Justification

### Task 1.1: Choose Your Framework

**Instructions:**
1. Review the 4 frameworks you learned
2. Select ONE framework for this challenge
3. Write a justification (150-200 words) explaining:
   - Why you chose this framework
   - What strengths make it suitable for this challenge
   - What trade-offs you considered
   - How its features align with the challenge requirements

**Available Frameworks:**
- CrewAI: Role-based agents, sequential/hierarchical processes
- LangGraph: Graph-based state management, conditional routing
- LlamaIndex: Data-centric, built-in RAG capabilities
- smolagents: Lightweight, tool-focused, minimal dependencies

---

### ✍️ YOUR FRAMEWORK SELECTION

**Selected Framework:** [WRITE YOUR CHOICE HERE]

**Justification:**

[WRITE YOUR JUSTIFICATION HERE - 150-250 words]

---

## 🛠️ Part 2: Setup & Configuration

### Task 2.1: Install Dependencies

Install your chosen framework and configure your API keys.

In [ ]:
# TODO: Install your chosen framework and dependencies
# Your code here:

### Task 2.2: Configure API Keys & Model

In [ ]:
# TODO: Configure your API key and model
# Your code here:

---

## 🏗️ Part 3: Implementation

Build your **Content Moderation & Enhancement System** by implementing the following components:

### System Architecture

```
User Content Input
      |
      v
┌─────────────────┐
│  Router Agent   │ ──> Classify: Social Media / Article / Comment
└────────┬────────┘
         |
         v
┌─────────────────────────┐
│ Parallel Analysis       │
│  - Safety Check Agent   │ ──> Detect harmful content
│  - Tone Analyzer Agent  │ ──> Assess sentiment/tone
│  - Grammar Checker      │ ──> Identify language issues
└────────┬────────────────┘
         |
         v
┌─────────────────────────┐
│ Evaluator Agent         │ ──> Aggregate findings, score content
└────────┬────────────────┘
         |
    ┌────┴─────┐
    v          v
  REJECT    APPROVE
            |
            v
    ┌───────────────┐
    │  Optimizer    │ ──> Suggest improvements
    │  Agent        │
    └───────┬───────┘
            |
            v
    ┌───────────────┐
    │  Enhancer     │ ──> Apply improvements
    │  Agent        │
    └───────┬───────┘
            |
            v
    Final Enhanced Content
```

---

### Task 3.1: Router Agent (Routing Pattern)

**Requirements:**
- Create a router that classifies content into: "social_media", "article", or "comment"
- Route should be based on length, structure, and style
- Return the classification decision

In [ ]:
# TODO: Implement Router Agent
# This agent analyzes content and classifies it

# Your code here:

### Task 3.2: Parallel Analysis Agents (Parallelization Pattern)

**Requirements:**
- Implement 3 agents that run in parallel:
  1. **Safety Checker**: Detect toxic, harmful, or inappropriate content
  2. **Tone Analyzer**: Assess sentiment (positive/negative/neutral) and professionalism
  3. **Grammar Checker**: Identify spelling, grammar, and clarity issues
- Each agent should return a structured assessment
- Execute them concurrently for efficiency

In [ ]:
# TODO: Implement Parallel Analysis Agents
# These agents analyze different aspects simultaneously

# Your code here:

### Task 3.3: Evaluator Agent (Evaluator-Optimizer Pattern - Part 1)

**Requirements:**
- Aggregate results from the 3 parallel agents
- Calculate an overall content quality score (0-100)
- Make a decision: APPROVE (score ≥ 70) or REJECT (score < 70)
- For approved content, provide specific improvement suggestions

In [ ]:
# TODO: Implement Evaluator Agent
# This agent aggregates findings and makes decisions

# Your code here:

### Task 3.4: Optimizer & Enhancer Agents (Prompt Chaining + Evaluator-Optimizer)

**Requirements:**
- **Optimizer Agent**: Generate specific improvements based on evaluator feedback
- **Enhancer Agent**: Apply improvements to create an enhanced version
- Implement as a chain: Original Content → Optimizer → Enhancer → Final Content
- (Optional) Add a re-evaluation loop if initial enhancement score is still low

In [ ]:
# TODO: Implement Optimizer and Enhancer Agents
# These agents improve content based on feedback

# Your code here:

### Task 3.5: Orchestrator (Orchestrator-Worker Pattern)

**Requirements:**
- Create a master orchestrator that coordinates the entire pipeline:
  1. Route content type
  2. Run parallel analysis
  3. Evaluate and decide
  4. If approved, optimize and enhance
  5. Return final result with metadata
- Handle both approval and rejection cases
- Provide clear logging of each step

In [ ]:
# TODO: Implement Orchestrator
# This coordinates the entire moderation pipeline

# Your code here:

---

## 🧪 Part 4: Testing

### Task 4.1: Test with Sample Content

Test your system with the provided examples representing different scenarios.

In [ ]:
# Test Case 1: Clean social media post (should be approved and enhanced)
test_content_1 = """
just finished reading an amzing book about AI ethics! 
its really make me think about how we build responsible systems. 
highly recomend it to anyone in tech!
"""

# Test Case 2: Professional article excerpt (should be approved, might need minor fixes)
test_content_2 = """
Machine learning algorithms have transformed the healthcare industry over the past decade.
These systems now assist in diagnosis, treatment planning, and patient monitoring.
However, concerns about data privacy and algorithmic bias remain significant challenges
that researchers and practitioners must address to ensure equitable healthcare delivery.
"""

# Test Case 3: Short comment with grammar issues (should be approved but needs enhancement)
test_content_3 = "this is grate! i totally agree with ur point about ai safety its so important"

# Test Case 4: Content with potential safety issues (might be rejected or flagged)
test_content_4 = """
I hate this stupid product! Complete waste of money. 
The company is terrible and everyone should avoid them.
"""

# TODO: Run your orchestrator on each test case
# Display the results clearly showing:
# - Content type classification
# - Analysis results (safety, tone, grammar)
# - Evaluation score and decision
# - Enhanced version (if approved)

print("="*70)
print("TEST CASE 1: Social Media Post with Errors")
print("="*70)
# Your test code here

print("\n" + "="*70)
print("TEST CASE 2: Professional Article")
print("="*70)
# Your test code here

print("\n" + "="*70)
print("TEST CASE 3: Short Comment")
print("="*70)
# Your test code here

print("\n" + "="*70)
print("TEST CASE 4: Potentially Problematic Content")
print("="*70)
# Your test code here

---

## 📊 Part 5: Reflection & Analysis

### Task 5.1: Pattern Usage Documentation

Document how you used each agentic pattern in your implementation.

### ✍️ YOUR PATTERN USAGE ANALYSIS

**1. Routing Pattern:**
- Where used: [DESCRIBE]
- Why effective: [EXPLAIN]

**2. Parallelization Pattern:**
- Where used: [DESCRIBE]
- Why effective: [EXPLAIN]
- Performance benefit: [EXPLAIN]

**3. Evaluator-Optimizer Pattern:**
- Where used: [DESCRIBE]
- How feedback loop works: [EXPLAIN]

**4. Prompt Chaining Pattern:**
- Where used: [DESCRIBE]
- Stages in chain: [LIST]

**5. Orchestrator-Worker Pattern:**
- How orchestration is managed: [DESCRIBE]
- Worker coordination: [EXPLAIN]

---

### Task 5.2: Challenges & Solutions

Reflect on difficulties you encountered and how you solved them.

### ✍️ YOUR CHALLENGES & SOLUTIONS

**Challenge 1:**
- Problem: [DESCRIBE]
- Solution: [EXPLAIN]

**Challenge 2:**
- Problem: [DESCRIBE]
- Solution: [EXPLAIN]

**Challenge 3:**
- Problem: [DESCRIBE]
- Solution: [EXPLAIN]

---

### Task 5.3: Framework Reflection

Now that you've completed the challenge, reflect on your framework choice.

### ✍️ YOUR FRAMEWORK REFLECTION

**What worked well with your chosen framework?**

[YOUR RESPONSE HERE]

**What was difficult or limiting?**

[YOUR RESPONSE HERE]

**Would you choose the same framework again? Why or why not?**

[YOUR RESPONSE HERE]

**What would you do differently next time?**

[YOUR RESPONSE HERE]

---

## 🎁 Bonus Challenges (Optional)

If you want to go further, try these enhancements:

### Bonus 1: Multi-Language Support
- Add a language detection agent
- Support content in at least 3 languages

### Bonus 2: Customizable Moderation Rules
- Allow users to set content policy preferences
- Adjust safety thresholds based on use case (e.g., strict for children's content)

### Bonus 3: Performance Optimization
- Measure execution time for each component
- Implement caching for repeated content
- Optimize parallel execution

### Bonus 4: Explainability Dashboard
- Create a visualization showing:
  - Agent decision flow
  - Confidence scores at each stage
  - Before/after content comparison

### Bonus 5: Iterative Re-evaluation
- If enhanced content scores < 90, run another optimization loop
- Limit to maximum 3 iterations to prevent infinite loops

---

In [ ]:
# Optional: Implement your bonus challenges here

---

## 📝 Evaluation Criteria

Your implementation will be assessed on:

### Functionality (20 points)
- ✅ Router correctly classifies content types
- ✅ Parallel agents execute concurrently
- ✅ Evaluator makes appropriate approve/reject decisions
- ✅ Enhancement chain improves content quality
- ✅ Orchestrator coordinates full pipeline

### Pattern Implementation (20 points)
- ✅ Routing pattern clearly implemented
- ✅ Parallelization working correctly
- ✅ Evaluator-optimizer feedback loop functional
- ✅ Prompt chaining evident in enhancement
- ✅ Orchestrator-worker hierarchy clear

### Code Quality (20 points)
- ✅ Clean, readable code
- ✅ Proper error handling
- ✅ Good documentation/comments
- ✅ Framework best practices followed

### Reflection & Analysis ( **40 points** )
- ✅ Thoughtful framework justification
- ✅ Clear pattern usage documentation
- ✅ Honest challenge/solution discussion
- ✅ Insightful framework reflection

### Bonus Points (up to 10 extra points)
- Optional challenges attempted and completed

---

## 🎉 Conclusion

Congratulations on completing this challenge! You've built a sophisticated multi-agent system that combines multiple agentic patterns in a real-world scenario.

### Key Takeaways

Through this challenge, you've learned:
- How to select appropriate frameworks for specific tasks
- How to combine multiple agentic patterns effectively
- How to design complex multi-agent systems
- How to handle real-world challenges in agent development
- How to evaluate and reflect on your architectural decisions

### Next Steps

1. **Experiment**: Try implementing this challenge with a different framework
2. **Extend**: Add more sophisticated features (RAG, custom tools, memory)
3. **Deploy**: Consider how you'd productionize this system
4. **Share**: Document your learnings and share with the community

Keep building, keep learning, and keep pushing the boundaries of what's possible with agentic systems! 🚀

---

**Happy Coding!** 💻✨